# 缶つまハンズオン マスターSQL

このノートブックは以下の4つのステップを順番に実行します：
- **Step1**: 画像からAIで情報を抽出してテーブルに格納
- **Step2**: Cortex Search サービスの作成
- **Step3**: Cortex Analyst用セマンティックビューの作成
- **Step4**: Cortex Agent の作成


---
## Step1: 画像からAIで情報を抽出してテーブルに格納

### 環境設定


In [ ]:
USE DATABASE HANDSON;
USE SCHEMA RAW_DATA;
USE WAREHOUSE COMPUTE_WH;


### 1-1. AI抽出結果格納テーブルの作成


In [ ]:
CREATE OR REPLACE TABLE HANDSON.RAW_DATA.PRODUCT_AI_EXTRACT (
    FILE_PATH             VARCHAR(500),
    FILE_NAME             VARCHAR(255),
    AI_DETAILED_CAPTION   VARCHAR(16777216),
    EXTRACT_JSON          VARIANT,
    CREATED_AT            TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP()
);


### 1-2. AIで抽出した結果をテーブルに挿入


In [ ]:
INSERT INTO HANDSON.RAW_DATA.PRODUCT_AI_EXTRACT (
    FILE_PATH,
    FILE_NAME,
    AI_DETAILED_CAPTION,
    EXTRACT_JSON
)
SELECT 
    '@HANDSON.RAW_DATA.HANDSON_RESOURCES/' || RELATIVE_PATH AS FILE_PATH,
    RELATIVE_PATH AS FILE_NAME,
    AI_COMPLETE(
        'llama4-maverick',
        PROMPT(
            '何が写っているのか説明をしてください: {0}', 
            TO_FILE('@HANDSON.RAW_DATA.HANDSON_RESOURCES', RELATIVE_PATH)
        )
    ) AS AI_DETAILED_CAPTION,
    AI_EXTRACT(
        file => TO_FILE('@HANDSON.RAW_DATA.HANDSON_RESOURCES', RELATIVE_PATH),
        responseFormat => {
            'product_name': '商品名は何ですか？(K&K 缶つまは除く)',
            'description':  '商品説明を教えて？',
            'jan_code':     'JANコードを教えて',
            'content':      '内容量(g)を教えてください',
            'Price':        '参考売価(円)を教えてください',
            'Ingredient':   '原材料を教えてください。カンマ区切りでリストアップしてください',
            'allergy':      'アレルギー成分を教えてください。カンマ区切りでリストアップしてください'
        }
    ) AS EXTRACT_JSON
FROM DIRECTORY(@HANDSON.RAW_DATA.HANDSON_RESOURCES)
WHERE RELATIVE_PATH LIKE '%.png';


### 結果の確認


In [ ]:
SELECT * FROM HANDSON.RAW_DATA.PRODUCT_AI_EXTRACT;


### 1-3. ANALYTICSスキーマに展開済みテーブルを作成


In [ ]:
USE SCHEMA ANALYTICS;


In [ ]:
CREATE OR REPLACE TABLE HANDSON.ANALYTICS.PRODUCT_MASTER (
    FILE_NAME             VARCHAR(255),
    PRODUCT_NAME          VARCHAR(500),
    AI_DETAILED_CAPTION   VARCHAR(16777216),
    DESCRIPTION           VARCHAR(16777216),
    JAN_CODE              VARCHAR(50),
    CONTENT               VARCHAR(100),
    PRICE                 VARCHAR(100),
    INGREDIENT            VARCHAR(16777216),
    ALLERGY               VARCHAR(16777216)
);


### 1-4. JSONを展開してANALYTICSテーブルに挿入


In [ ]:
INSERT INTO HANDSON.ANALYTICS.PRODUCT_MASTER
SELECT 
    FILE_NAME,
    EXTRACT_JSON:response:product_name::VARCHAR                    AS PRODUCT_NAME,
    AI_DETAILED_CAPTION                                            AS AI_DETAILED_CAPTION,
    EXTRACT_JSON:response:description::VARCHAR                     AS DESCRIPTION,
    EXTRACT_JSON:response:jan_code::VARCHAR                        AS JAN_CODE,
    EXTRACT_JSON:response:content::VARCHAR                         AS CONTENT,
    EXTRACT_JSON:response:Price::VARCHAR                           AS PRICE,
    ARRAY_TO_STRING(EXTRACT_JSON:response:Ingredient::ARRAY, ', ') AS INGREDIENT,
    ARRAY_TO_STRING(EXTRACT_JSON:response:allergy::ARRAY, ', ')    AS ALLERGY
FROM HANDSON.RAW_DATA.PRODUCT_AI_EXTRACT;


---
## Step2: Cortex Search サービスの作成

商品マスタ用 Cortex Search サービス  
商品説明やAIキャプションで検索可能


In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE HANDSON.ANALYTICS.PRODUCT_SEARCH_SERVICE
    ON SEARCH_TEXT
    ATTRIBUTES JAN_CODE, PRODUCT_NAME, FILE_NAME, CONTENT, PRICE, ALLERGY
    WAREHOUSE = COMPUTE_WH
    TARGET_LAG = '1 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
    SELECT
        JAN_CODE,
        PRODUCT_NAME,
        FILE_NAME,
        DESCRIPTION,
        AI_DETAILED_CAPTION,
        CONTENT,
        PRICE,
        INGREDIENT,
        ALLERGY,
        -- 検索対象テキスト（商品名、説明、AIキャプション、原材料を結合）
        CONCAT(
            COALESCE(PRODUCT_NAME, ''),        ' ',
            COALESCE(AI_DETAILED_CAPTION, ''), ' ',
            COALESCE(INGREDIENT, ''),
            COALESCE(ALLERGY, '')
        ) AS SEARCH_TEXT
    FROM HANDSON.ANALYTICS.PRODUCT_MASTER
);


### Cortex Search サービスの確認


In [ ]:
SHOW CORTEX SEARCH SERVICES IN SCHEMA HANDSON.ANALYTICS;


---
## Step3: Cortex Analyst用セマンティックビューの作成

YAMLで定義されたセマンティックモデル「KANTSUMA_ANALYST」を  
Snowflakeのセマンティックビューとして実装します


### 3-1. 前準備: RAW_DATAスキーマのテーブルをANALYTICSスキーマにビューとして作成

セマンティックビューでは同一スキーマ内のテーブル/ビューを参照する必要があるため


In [ ]:
CREATE OR REPLACE VIEW INVENTORY AS
SELECT * FROM HANDSON.RAW_DATA.INVENTORY;


In [ ]:
CREATE OR REPLACE VIEW SALES AS
SELECT * FROM HANDSON.RAW_DATA.SALES;


### 3-2. セマンティックビュー（Semantic View）の定義

Snowflakeネイティブのセマンティックビュー機能を使用


In [ ]:
CREATE OR REPLACE SEMANTIC VIEW KANTSUMA_ANALYST

    -- ========================================
    -- テーブル定義
    -- ========================================
    tables (
        INVENTORY AS INVENTORY 
            PRIMARY KEY (JAN_CODE)
            WITH SYNONYMS = ('在庫', '棚卸', '在庫データ', '棚卸データ', '在庫レベル')
            COMMENT = '在庫スナップショット記録。特定日付における製品の在庫レベル、数量、単価、総在庫評価額を含む。',

        PRODUCT_MASTER AS PRODUCT_MASTER 
            PRIMARY KEY (JAN_CODE)
            WITH SYNONYMS = ('商品マスタ', '商品情報', '商品詳細', 'マスタ', '商品', '製品', '製品マスタ')
            COMMENT = '商品マスタデータ。説明、価格、内容物、原材料、アレルギー警告などの情報を含む。',

        SALES AS SALES 
            PRIMARY KEY (JAN_CODE)
            WITH SYNONYMS = ('売上', '販売', '売上データ', '販売データ', '取引', '売上取引')
            COMMENT = '販売取引記録。製品詳細、価格情報、取引日時を含む。'
    )

    -- ========================================
    -- リレーションシップ定義
    -- ========================================
    relationships (
        JAN_INVENT AS PRODUCT_MASTER(JAN_CODE) REFERENCES INVENTORY(JAN_CODE),
        JAN_SALES  AS PRODUCT_MASTER(JAN_CODE) REFERENCES SALES(JAN_CODE)
    )

    -- ========================================
    -- ファクト定義（数値データ）
    -- ========================================
    facts (
        -- 在庫関連
        INVENTORY.inventory_quantity_value AS INVENTORY_QUANTITY
            COMMENT = '在庫数量。在庫レベルの集計に使用',
        INVENTORY.inventory_value_amount   AS INVENTORY_VALUE
            COMMENT = '在庫金額（円）',
        INVENTORY.unit_cost_value          AS UNIT_COST
            COMMENT = '単価（円）',

        -- 売上関連
        SALES.sales_quantity      AS QUANTITY
            COMMENT = '販売数量。販売個数の集計に使用',
        SALES.sales_amount_value  AS SALES_AMOUNT
            COMMENT = '売上金額（円）',
        SALES.unit_price_value    AS UNIT_PRICE
            COMMENT = '販売単価（円）'
    )

    -- ========================================
    -- ディメンション定義（属性データ）
    -- ========================================
    dimensions (
        -- INVENTORY ディメンション
        INVENTORY.jan_code AS JAN_CODE
            WITH SYNONYMS = ('JANコード', '商品コード', '製品コード', 'バーコード')
            COMMENT = 'Japanese Article Number codes used for product identification.',
        INVENTORY.product_name AS PRODUCT_NAME
            WITH SYNONYMS = ('商品名', '製品名', '品名')
            COMMENT = 'Japanese food product names describing various grilled meats, seafood, and regional specialties.',
        INVENTORY.inventory_date AS INVENTORY_DATE
            WITH SYNONYMS = ('在庫日', '棚卸日', '記録日')
            COMMENT = 'The date when inventory was recorded or measured.',
        INVENTORY.inventory_year AS YEAR(inventory_date)
            WITH SYNONYMS = ('在庫年', '棚卸年')
            COMMENT = '在庫が記録された年',
        INVENTORY.inventory_month AS MONTH(inventory_date)
            WITH SYNONYMS = ('在庫月', '棚卸月')
            COMMENT = '在庫が記録された月',

        -- PRODUCT_MASTER ディメンション
        PRODUCT_MASTER.jan_code AS JAN_CODE
            WITH SYNONYMS = ('JANコード', '商品コード', '製品コード', 'バーコード')
            COMMENT = 'Japanese Article Number codes used for product identification.',
        PRODUCT_MASTER.product_name AS PRODUCT_NAME
            WITH SYNONYMS = ('商品名', '製品名', '品名')
            COMMENT = 'Japanese food product names.',
        PRODUCT_MASTER.description AS DESCRIPTION
            WITH SYNONYMS = ('商品説明', '説明', '詳細説明', '商品詳細')
            COMMENT = 'Product descriptions in Japanese detailing ingredients, preparation methods, and flavor characteristics of various food items.',
        PRODUCT_MASTER.ingredient AS INGREDIENT
            WITH SYNONYMS = ('原材料', '材料', '成分', '原材料名')
            COMMENT = 'Product ingredients and allergen information listed in Japanese.',
        PRODUCT_MASTER.allergy AS ALLERGY
            WITH SYNONYMS = ('アレルギー', 'アレルギー情報', 'アレルゲン', 'アレルギー物質')
            COMMENT = 'Food allergens and ingredients that may cause allergic reactions.',
        PRODUCT_MASTER.content AS CONTENT
            WITH SYNONYMS = ('内容量', '容量', '内容')
            COMMENT = 'The content or substance of the product.',
        PRODUCT_MASTER.price AS PRICE
            WITH SYNONYMS = ('価格', '売価', '参考売価', '定価')
            COMMENT = 'The price of the product.',
        PRODUCT_MASTER.ai_detailed_caption AS AI_DETAILED_CAPTION
            WITH SYNONYMS = ('AI説明', 'AI詳細', '画像説明', 'AIキャプション')
            COMMENT = 'Detailed artificial intelligence-generated captions describing product images and their contents.',
        PRODUCT_MASTER.file_name AS FILE_NAME
            WITH SYNONYMS = ('ファイル名', '画像ファイル', '画像パス')
            COMMENT = 'File paths to product image files stored in PNG format.',

        -- SALES ディメンション
        SALES.jan_code AS JAN_CODE
            WITH SYNONYMS = ('JANコード', '商品コード', '製品コード', 'バーコード')
            COMMENT = 'Japanese Article Number codes used for product identification.',
        SALES.product_name AS PRODUCT_NAME
            WITH SYNONYMS = ('商品名', '製品名', '品名')
            COMMENT = 'Japanese food product names describing various meat and seafood dishes.',
        SALES.sale_date AS SALE_DATE
            WITH SYNONYMS = ('売上日', '販売日', '取引日')
            COMMENT = 'The date when the sale transaction occurred.',
        SALES.sale_year AS YEAR(sale_date)
            WITH SYNONYMS = ('売上年', '販売年')
            COMMENT = '売上が発生した年',
        SALES.sale_month AS MONTH(sale_date)
            WITH SYNONYMS = ('売上月', '販売月')
            COMMENT = '売上が発生した月'
    )

    -- ========================================
    -- メトリクス定義（集計関数）
    -- ========================================
    metrics (
        -- 売上メトリクス
        SALES.total_sales_amount   AS SUM(sales.sales_amount_value)
            WITH SYNONYMS = ('総売上金額', '合計売上金額', '売上合計')
            COMMENT = '売上金額の合計額',
        SALES.total_sales_quantity AS SUM(sales.sales_quantity)
            WITH SYNONYMS = ('総販売数量', '合計販売数量', '販売数合計')
            COMMENT = '販売数量の合計',
        SALES.average_unit_price   AS AVG(sales.unit_price_value)
            WITH SYNONYMS = ('平均単価', '平均販売単価')
            COMMENT = '販売単価の平均額',

        -- 在庫メトリクス
        INVENTORY.total_inventory_quantity AS SUM(inventory.inventory_quantity_value)
            WITH SYNONYMS = ('総在庫数量', '合計在庫数量', '在庫数合計')
            COMMENT = '在庫数量の合計',
        INVENTORY.total_inventory_value    AS SUM(inventory.inventory_value_amount)
            WITH SYNONYMS = ('総在庫金額', '合計在庫金額', '在庫金額合計')
            COMMENT = '在庫金額の合計額',
        INVENTORY.average_unit_cost        AS AVG(inventory.unit_cost_value)
            WITH SYNONYMS = ('平均単価', '平均原価')
            COMMENT = '単価の平均額'
    )

    COMMENT = '缶つま商品の売上・在庫・商品マスタに関する分析のためのセマンティックビュー';


---
## Step4: Cortex Agent の作成

Cortex SearchとCortex Analystを統合したエージェント


In [ ]:
CREATE OR REPLACE AGENT KANTSUMA_AGENT
    COMMENT = '缶つま商品の売上・在庫分析と商品検索を行うエージェント'
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": ""
  },
  "instructions": {
    "response": "あなたは「缶つま」ブランドの商品データアナリストです。ユーザーからの質問に対して、売上データ、在庫データ、商品情報を活用して回答してください。数値データは適切にフォーマットして表示してください（例：金額は円表記、数量は個数表記）。商品名は正確に記載してください。分析結果には根拠となるデータを含めてください。日本語で丁寧に回答してください。可能であれば視覚化を提供してください。トレンドラインは線グラフをデフォルトとし、カテゴリは棒グラフを使用してください。",
    "orchestration": "商品の詳細情報（説明、原材料、アレルギー情報など）を検索する場合はcortex searchを使用し、売上や在庫の数値分析にはcortex analystを使用してください。複合的な質問の場合は、まずcortex searchで商品を特定し、その結果をcortex analystに渡して分析してください。",
    "sample_questions": [
      {
        "question": "缶つま商品の2025年の売上状況を可視化してほしい。急激に売り上げが上昇した商品も教えて欲しい。"
      },
      {
        "question": "これまでの傾向を踏まえてそれらの商品は2026年1月でどのくらい在庫を抱えれば良いと思いますか？"
      },
      {
        "question": "来月のフェアで、『健康志向（低糖質・高タンパク）』かつ『アレルギー品目”小麦”を含まない』新しいおつまみ缶詰を提案してほしい。ただし、現在在庫があって即納できるものに限る。"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "KANTSUMA_ANALYST",
        "description": "缶つま商品の売上データ、在庫データ、商品マスタをクエリして分析することができます。売上金額、販売数量、在庫数量、在庫金額などの集計・分析に使用します。"
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "PRODUCT_SEARCH",
        "description": "缶つま商品の詳細情報を検索します。商品名、商品説明、原材料、アレルギー情報などのテキスト検索に使用します。"
      }
    }
  ],
  "tool_resources": {
    "KANTSUMA_ANALYST": {
      "semantic_view": "HANDSON.ANALYTICS.KANTSUMA_ANALYST"
    },
    "PRODUCT_SEARCH": {
      "name": "HANDSON.ANALYTICS.PRODUCT_SEARCH_SERVICE",
      "max_results": 5
    }
  }
}
$$;